In [2]:
import pandas as pd
import psycopg2 as psql

In [3]:
# Load in the Data
train = pd.read_json('train.json')
test = pd.read_json('test.json')

In [43]:
len(train)

39774

In [41]:
def dbPut(table, data, db = 'postgres', user = 'postgres'):
    
    """ Connect to postgres database, create table (drop if exists), add column with n-rows to database.
    Train is expected to be a pandas dataframe. """
    
    colNames = list(data.columns.values)

    connect_text = "dbname=%s user=%s" % (db, user)
    conn = psql.connect(connect_text)
    
    cur = conn.cursor()
    
    drop_text = "DROP TABLE IF EXISTS %s;" % (table, )
    cur.execute(drop_text)

    create_text = 'CREATE TABLE %s (%s TEXT, %s INTEGER, %s TEXT); ' % (table, colNames[0], colNames[1], colNames[2])
    
    cur.execute(create_text)
    
    addthis = 'INSERT INTO %s (%s, %s, %s)' \
            ' VALUES (%%s, %%s, %%s);' \
            % (table, colNames[0], colNames[1], colNames[2])

    for i in range(len(data)):
        cur.execute(addthis, (data.iloc[i][0], int(data.iloc[i][1]), data.iloc[i][2]))
    
    conn.commit()
    
    cur.close()
    conn.close()

In [42]:
db = 'postgres'
user = 'postgres'
table = 'cookingTrain'
dbPut(table, train, db, user)

# Explore data

In [ ]:
# select * from cookingtrain
# where ingredients like '%fresh%';

# 13658 rows

In [ ]:
# select * from cookingtrain
# where ingredients like '%large%';

# 4902 rows